In [1]:
%matplotlib inline
import os
from osgeo import gdal
import rasterio
import geopandas as gpd
import rasterio.mask
from rasterio.windows import Window
import sys
from shapely.geometry import mapping
sys.path.append(r'E:/gitlab/geoloc2/Detecciondeterrenos')
from codigos import Generar_txt
###path de yolo dentro de computadora
os.chdir(r'C:/Users/ASUS/Inteligencia_Artificial/yolov7')
from detect_Alberto_v4 import *
from scipy.ndimage import rotate as rotate_image
from shapely import geometry
import time
import datetime

In [2]:
torch.cuda.is_available()

True

## Modelo de deteccion de objetos basado en yolov7

In [3]:
Modelo=modelo(weights='Modelos/best_Fer_2.pt')#,device='cpu') ## device='0' para gpu

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



## Modelo clasificador basado en Alexnet

In [4]:
weights=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\ckpoint\best_modell7.pth"
num_classes=6
diciconario={0: 'carros', 1: 'casas', 2: 'en_construccion', 3: 'establecimiento', 4: 'multivivienda', 5: 'terreno_baldio'}
model_class=alexnet(weights=weights,num_classes=num_classes,idx_to_class=diciconario)

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 256, 6, 6]           --
|    └─Conv2d: 2-1                       [-1, 64, 55, 55]          23,296
|    └─Hardtanh: 2-2                     [-1, 64, 55, 55]          --
|    └─MaxPool2d: 2-3                    [-1, 64, 27, 27]          --
|    └─Conv2d: 2-4                       [-1, 192, 27, 27]         307,392
|    └─ReLU: 2-5                         [-1, 192, 27, 27]         --
|    └─MaxPool2d: 2-6                    [-1, 192, 13, 13]         --
|    └─Conv2d: 2-7                       [-1, 384, 13, 13]         663,936
|    └─ReLU: 2-8                         [-1, 384, 13, 13]         --
|    └─Conv2d: 2-9                       [-1, 256, 13, 13]         884,992
|    └─ReLU: 2-10                        [-1, 256, 13, 13]         --
|    └─Conv2d: 2-11                      [-1, 256, 13, 13]         590,080
|    └─ReLU: 2-12                        [-1, 256, 13, 13]   

In [5]:
## test clasificador sobre archivo
file=r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\validacion_pad\casas\0_10.png"
print(model_class.predict_file(file,pad=False))
##test clasificador sobre imagen precargada
image=cv2.imread(file)
print(model_class.predict_image(image,pad=False)[0])

establecimiento
establecimiento


In [6]:
# import torch #or pytorch, works for the same shit and contains the same libraries that you need such as Numpy (as far as I know)
# from sklearn.metrics import confusion_matrix
# y_true = []
# y_predict =[]
# import glob
# filenames=glob.glob(r"C:\Users\ASUS\Inteligencia_Artificial\clasificador\train_pad/*/*")
# for file in tqdm.tqdm(filenames):
#     file=file.replace("\\","/")
#     y_true.append(file.split("/")[-2])
#     y_predict.append(model_class.predict_file(file,pad=False))
# labels=[]
# for k,v in diciconario.items():
#     labels.append(v)
# labels
# confusion_mat = confusion_matrix(y_true, y_predict,labels=labels)
# conteos=pd.DataFrame(y_true,columns=["clases"]).value_counts()
# conteos=conteos.reset_index(drop=False)
# matrix=[]
# for i,label in enumerate(labels):
#     matrix.append(confusion_mat[i]/conteos[conteos["clases"]==label][0].values[0])
# import matplotlib.pyplot as plt

# plt.imshow(matrix)
# matrix

## Parametros del raster

In [7]:
raster=r"C:\Users\ASUS\Desktop\nauc.tif"
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy=Parametro_raster(raster,metros=60)
alto,ancho,dim,crs,H,W,minx,maxx,miny,maxy

(243,
 381,
 121,
 CRS.from_epsg(3857),
 29456,
 46105,
 -11066760.9206,
 -11043708.4278,
 2203484.4013,
 2218212.1729)

## Delimitacion por shape de municipio

In [8]:
shape_o=gpd.read_file(r'C:/Users/ASUS/Documents/GobiernoEdomex/Agua/geoshapes/Full manzanas/Manzana_Naucalpan.shp')
shape_o=shape_o.to_crs('3857')
shape=shape_transform(shape_o.copy())
#shape_aumentado=ampliar_shape(shape_o.copy())

## Generar prediccion sobre raster(municipio) con salida en shp

In [ ]:
nombres=[]
imshow=False
result=pd.DataFrame()
casas=[]
terreno=[]
angulosget=[]
conf_casas=[]
conf_terreno=[]
clase_casas=[]
clase_terreno=[]
s=0

### Impementar modelos combinados

In [ ]:
postproceso(Modelo,model_class,casas,conf_casas,clase_casas,terreno,conf_terreno,clase_terreno,raster,ancho,alto,dim,minx,maxx,miny,maxy,shape,angulo_get=0,opt_conf_thres=0.02,imshow=False,imsave=True,path="C:/Users/ASUS/salida/imagen")

In [ ]:
gdf_casas=gpd.GeoDataFrame({"clase_detectada":clase_casas,"conf":conf_casas},geometry=casas,crs=crs)
gdf_casas.set_crs=crs
gdf_casas["area"]=gdf_casas.area
gdf_casas=gdf_casas.astype({"conf":"float64"})
gdf_casas.to_file(r"C:/Users/ASUS/salida/shape/naucalpan_casas_nuevomodelo5.shp")

In [ ]:
gdf_terreno=gpd.GeoDataFrame({"clase_detectada":clase_terreno,"conf":conf_terreno},geometry=terreno,crs=crs)
gdf_terreno.set_crs=crs
gdf_terreno["area"]=gdf_terreno.area
gdf_terreno=gdf_terreno.astype({"conf":"float64"})
gdf_terreno.to_file(r"C:/Users/ASUS/salida/shape/naucalpan_terrenos_nuevomodelo5.shp")

In [ ]:
ñ ##breakpoint

## Generar imagenes mosaico

In [ ]:
# Path donde se guardan images
path_mosaico_salida=r"D:\instream\Mi unidad\geoshaps\neza\images/"
generar_imagenes=int(input("Generar imagenes: 1 si, 0 no "))
if generar_imagenes==1:
    generar_imagenes_sinrotar=int(input("Generar imagenes sin rotar: 1 si, 0 no "))
    generar_imagenes_rotadas=int(input("Generar imagenes rotadas: 1 si, 0 no "))
    with tqdm.tqdm(total=alto*ancho) as pbar:
        for j in range(0,ancho):#ancho
            for i in (range(alto)):#alto
                generar=0
                label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
                nameimg=label.lower()+str(i)+"_"+str(j)
                cuadro=[]
                for k in range(2):
                    for l in range(2):
                        cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                                       maxy-(maxy-miny)/alto*(i+l),
                                       0.0))
                cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
                for punto in cuadro:
                    x=float(punto[0])
                    y=float(punto[1])
                    if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                        generar=1             
                if generar==1:
                    shapes=[{"type":'Polygon','coordinates':[cuadro]}]
                    array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
                    four_images=[array[2],array[1],array[0]]
                    imagen_n = np.stack(four_images, axis=-1)
                    if generar_imagenes_sinrotar==1:
                        cv2.imwrite(path_mosaico_salida+nameimg+".png",imagen_n)
                    if generar_imagenes_rotadas==1:
                        angulo_1,imagen_ro=correct_orientation(imagen_n,dim)
                        cv2.imwrite(path_mosaico_salida+nameimg+"_"+str(angulo_1)+".png",imagen_ro)
                pbar.update(1)

## Generar prediccion sobre imagen con salida txt

In [ ]:
# path_mosaico_salida="D:/tlalne/"
vector=Modelo.detect(opt_source=path_mosaico_salida,opt_conf_thres=0.08)
Generar_txt(vector,path_mosaico_salida+"labels")
with open(path_mosaico_salida+"labels/classes.txt","w") as f:
    f.writelines("\n".join(["casa","terreno_baldio"]))

In [ ]:
## shape igecem de manzanas por municipio (solo pora tener la delimitacion geografica del municipio)
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Naucalpan.shp")
shape=shape.to_crs("3857")
c=[]
for manzana in range(len(shape)):
    proyecciones1=mapping(shape["geometry"][manzana]).get("coordinates")
    if len(proyecciones1)>1:
        for manzan in proyecciones1:
            print(manzana)
            try:
                proyecciones=manzan
            except:
                proyecciones=manzan[0]
            point1=np.min((proyecciones,proyecciones),axis=1)[0]
            min_y,min_x=point1[0],point1[1]
            point2=np.max((proyecciones,proyecciones),axis=1)[0]
            max_y,max_x=point2[0],point2[1]
            c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
    else:
        proyecciones=proyecciones1[0]
        point1=np.min((proyecciones,proyecciones),axis=1)[0]
        min_y,min_x=point1[0],point1[1]
        point2=np.max((proyecciones,proyecciones),axis=1)[0]
        max_y,max_x=point2[0],point2[1]
        c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones    
    
    # point1=np.min((proyecciones,proyecciones),axis=1)[0]
    # min_y,min_x=point1[0],point1[1]
    # point2=np.max((proyecciones,proyecciones),axis=1)[0]
    # max_y,max_x=point2[0],point2[1]
    # c.append(",".join([str(min_y),str(min_x),str(max_y),str(max_x)]))#,proyecciones
shape1=pd.DataFrame()
shape1["points"]=c
shape1=shape1["points"].str.split(",",expand=True)
shape1=shape1.astype({0:"float64",1:"float64",2:"float64",3:"float64"})
shape=shape1
shape

In [ ]:
from shapely.geometry import Point
geo=[]
shape=gpd.read_file(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\geoshapes\Full manzanas\Manzana_Naucalpan.shp")
shape=shape.to_crs("3857")
c=[]
for manzana in range(len(shape)):
    proyecciones1=mapping(shape["geometry"][manzana]).get("coordinates")
    for i in proyecciones1[0]:
        geo.append(Point(i))
sa=gpd.GeoDataFrame(geometry=geo)
sa

In [ ]:
sa.to_file(r"C:\Users\ASUS\Desktop\nau\pointnau.shp")

In [ ]:
gpd.sjoin(shape,shape1 , how='left', op='intersects').to_file(r"C:\Users\ASUS\Desktop\nau\sol.shp")

In [ ]:
shape1=gpd.read_file(r"C:\Users\ASUS\Desktop\nau\new_terreno1.shp")z
shape1

In [ ]:
indices=[]
for i,pol in enumerate(shape1["geometry"]):
    generar=0
    cuadro=mapping(pol).get("coordinates")[0]
    for punto in cuadro:
            x=float(punto[0])
            y=float(punto[1])
            if len(shape[(shape[0]<=x)&(shape[2]>=x)&(shape[1]<=y)&(shape[3]>=y)])>0:
                generar=1             
    if generar==1:
         indices.append(i)
indices

In [ ]:
shape1.loc[indices].to_file(r"C:\Users\ASUS\Desktop\nau/solo_naucalpan.shp")

In [ ]:
gpd.sjoin(shape,shape1 , how='inner', op='intersects').to_file(r"C:\Users\ASUS\Desktop\nau/sol.shp")

In [ ]:
complete_polys = [polygon for polygon in  if (polygon.centroid.x**2 + polygon.centroid.y**2 < 4**2)]

complete_polys

In [ ]:
import random
import rasterio
from rasterio.windows import Window

with rasterio.open(r"D:\haar\classes\carros.tif") as src:

    # The size in pixels of your desired window
    xsize, ysize = 1000,1000

    # Generate a random window origin (upper left) that ensures the window 
    # doesn't go outside the image. i.e. origin can only be between 
    # 0 and image width or height less the window width or height
    xmin, xmax = 0, src.width - xsize
    ymin, ymax = 0, src.height - ysize
    xoff, yoff = random.randint(xmin, xmax), random.randint(ymin, ymax)

    # Create a Window and calculate the transform from the source dataset    
    window = Window(0, 0, xsize, ysize)
    transform = src.window_transform(window)

    # Create a new cropped raster to write to
    profile = src.profile
    profile.update({
        'height': xsize,
        'width': ysize,
        'transform': transform})

    with rasterio.open('D:/haar/classes/carros_output.tif', 'w', **profile) as dst:
        # Read the data from the window and write it to the output raster
        dst.write(src.read(window=window))